In [11]:
#import necessary libraries 
import cv2
import numpy as np
from tensorflow import keras

In [12]:
#loading the model using load_model from keras library
model = keras.models.load_model("./saved_model")
results={1:'without mask',0:'mask'}
GR_dict={0:(0,255,0),1:(0,0,255)}
rect_size = 4
# To capture video from webcam. 
cap = cv2.VideoCapture(0) 

#haar cascade classifiers is an object detection algorithm to detect faces in an image or real-time video
# Loading the cascade classifier
face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")

2022-03-19 20:51:45.439844: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-19 20:51:45.440101: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [13]:
#bounding box for the detection of face
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    #Detecting the face
    faces = face_cascade.detectMultiScale(rerect_size)
    
    # To draw the rectangle around each face
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Display
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

2022-03-19 20:51:48.670793: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-19 20:51:48.723802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [14]:
#release webcam 
cap.release()
#closing all windows
cv2.destroyAllWindows()